In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import pickle
import os

In [4]:
folder = "/home/kardosp/continuethelist/outputs/"
file = "kgvec2go_v1_output_l1ellipse_sample4.pickle"

In [5]:
with open(os.path.join(folder, file), "rb") as f:
    data = pickle.load(f)

In [6]:
type(data)

list

In [7]:
len(data)

250

In [8]:
data[2]

{'_id': '2099',
 'corrected_question': 'What is the region of the ethnic group which speaks the language of Arkansas?',
 'intermediary_question': 'What is the <region> of the <ethnic group> which is the <language> of <Arkansas> ?',
 'sparql_query': 'SELECT DISTINCT ?uri WHERE { <http://dbpedia.org/resource/Arkansas> <http://dbpedia.org/ontology/language> ?x . ?x <http://dbpedia.org/property/region> ?uri  . ?x <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/EthnicGroup>}',
 'sparql_template_id': 303,
 'result': ['http://dbpedia.org/resource/California',
  'http://dbpedia.org/resource/Florida',
  'http://dbpedia.org/resource/Georgia_(U.S._state)',
  'http://dbpedia.org/resource/Illinois',
  'http://dbpedia.org/resource/Michigan',
  'http://dbpedia.org/resource/New_York',
  'http://dbpedia.org/resource/North_Carolina',
  'http://dbpedia.org/resource/Ohio',
  'http://dbpedia.org/resource/Pennsylvania',
  'http://dbpedia.org/resource/Tennessee',
  'http://dbpe

In [19]:
mismatch_counter = 0
for i, row in enumerate(data):
    if row["output"] is None:
        continue
    if row["seed"] is None:
        continue
    if len(row["output"]) != len(row["seed"]):
        mismatch_counter += 1
        #print("OUTPUT: ", row["output"])
        #print("SEED", row["seed"])
        print(i, len(row["output"]), len(row["seed"]))

1 719 4
3 2 4
4 3 4
15 3 4
27 2 4
31 3 4
41 3 4
42 9 4
53 3 4
58 3 4
63 3 4
72 2 4
74 2 4
77 3 4
79 3 4
81 3 4
100 0 4
108 3 4
114 3 4
153 3 4
156 3 4
163 3 4
166 3 4
170 3 4
175 3 4
177 2 4
179 2 4
186 2 4
190 2 4
194 3 4
196 3 4
205 3 4
207 3 4
209 3 4
215 9989 4
219 3 4
220 3 4
228 3 4
233 3 4
234 3 4
235 3 4
245 0 4


In [20]:
data[3]

{'_id': '2451',
 'corrected_question': 'What are the other band member of the Bands whose current members is Hynief?',
 'intermediary_question': 'What is the <band member> of the <Bands> whose <current members> is <Hynief>>?',
 'sparql_query': 'SELECT DISTINCT ?uri WHERE { ?x <http://dbpedia.org/property/currentMembers> <http://dbpedia.org/resource/Hynief> . ?x <http://dbpedia.org/ontology/bandMember> ?uri  . ?x <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Band>}',
 'sparql_template_id': 305,
 'result': ['http://dbpedia.org/resource/Johnny_Depp',
  'http://dbpedia.org/resource/Robert_De_Niro',
  'http://dbpedia.org/resource/Juelz_Santana',
  'http://dbpedia.org/resource/Purple_City_Productions',
  "http://dbpedia.org/resource/Big_B'Z",
  'http://dbpedia.org/resource/Hynief',
  'http://dbpedia.org/resource/Nick_Haze',
  'http://dbpedia.org/resource/Richmond_Rab',
  'http://dbpedia.org/resource/Riq_Rose',
  'http://dbpedia.org/resource/SkullGizzy(Gamer)'

In [16]:
len(data[2]["output"])

4

In [17]:
len(data[2]["seed"])

4

In [11]:
mismatch_counter

42

In [15]:
from algorithm.kgvec2go_vectors_simple import run_method, load_gensim_model
from excludes.basic_excludes import (node_excl_yago_func, node_excl_extra,
                                     node_excl_owlthing_func, node_excl_wiki_func)
from loaders.loaders import *
from evaluation.split_dataset import split_on_seed_dataset
import json
import pickle
import os
from tqdm import tqdm


exclude_paths = [
    ["", "http://www.w3.org/1999/02/22-rdf-syntax-ns#type", "http://www.w3.org/2004/02/skos/core#Concept"]
]
relation_excludes = ["http://xmlns.com/foaf/0.1/primaryTopic",
                     "http://dbpedia.org/property/isCitedBy",
                     "http://dbpedia.org/ontology/wikiPageWikiLink",
                     "http://dbpedia.org/ontology/wikiPageWikiLinkText",
                     "http://dbpedia.org/property/wikiPageUsesTemplate",
                     'http://dbpedia.org/ontology/wikiPageRedirects',
                     'http://dbpedia.org/ontology/wikiPageOutDegree',
                     "http://dbpedia.org/ontology/abstract",
                     "http://www.w3.org/2000/01/rdf-schema#comment", ]

res_excludes_inside_point_relation = ["http://purl.org/linguistics/gold/hypernym",
                                      "http://www.w3.org/1999/02/22-rdf-syntax-ns#type"]
res_excludes_inside_point_node = []
node_excludes_funcs = [node_excl_wiki_func, node_excl_owlthing_func, node_excl_yago_func, node_excl_extra]

SEED = 44
SAMPLE_SIZE = 4
verbose = 1

#Dataset load
#lc_quad_train = load_lc_quad_train()

lc_quad_train = load_on_path(ds_lc_quad_train_cleaned)
splitted_lcquad = split_on_seed_dataset(lc_quad_train, SAMPLE_SIZE, SEED)

#splitted_lcquad = load_on_path_pickle("/home/kardosp/continuethelist/outputs/graphwalk_v2_output_sample4.pickle")

cwd = "/home/kardosp/continuethelist/dbpedia_vectors/kgvec2go"
distance_metric = "l1ellipse"
word2vec_path = os.path.join(cwd, 'model.kv')
wv_model = load_gensim_model(word2vec_path)
missing_counter = 0


In [16]:

# lcquad_output = run_on_dataset(splitted_lcquad, exclude_paths, relation_excludes, node_excludes_funcs)
for i, record in tqdm(enumerate(splitted_lcquad[:1])):
    print(i)
    if "output" in record:
        continue
    result, missing = run_method(record["seed"], wv_model, distance=distance_metric)
    record["output"] = result
    missing_counter += missing
    print(missing_counter)
    # TODO - evaluate
    print("result:")
    print(result)

0it [00:00, ?it/s]

0
(200,)
(4, 200)


1it [02:56, 176.80s/it]

0
result:
[]


In [17]:
folder = "/home/kardosp/continuethelist/outputs/"
file = "kgvec2go_v1_output_circle_sample4.pickle"

with open(os.path.join(folder, file), "rb") as f:
    data = pickle.load(f)


In [18]:
data[0]

{'_id': '2413',
 'corrected_question': 'Starwood operates in which places?',
 'intermediary_question': 'What is the <address> of the <hotels> whose <operator> is <Starwood Hotels and Resorts Worldwide>?',
 'sparql_query': 'SELECT DISTINCT ?uri WHERE { ?x <http://dbpedia.org/property/operator> <http://dbpedia.org/resource/Starwood_Hotels_and_Resorts_Worldwide> . ?x <http://dbpedia.org/property/address> ?uri  . }',
 'sparql_template_id': 5,
 'result': ['181',
  'http://dbpedia.org/resource/California',
  'http://dbpedia.org/resource/San_Francisco',
  '2',
  'Calle San Fernando, 2',
  '1',
  'http://dbpedia.org/resource/Chennai',
  'http://dbpedia.org/resource/Tamil_Nadu',
  'Krakowskie Przedmiescie 42/44',
  '1900',
  'http://dbpedia.org/resource/Seattle',
  '811',
  '333',
  '210',
  'http://dbpedia.org/resource/Atlanta',
  '10600',
  'http://dbpedia.org/resource/Westminster,_Colorado',
  'Main Gulshan Avenue, Plot-01, Road 45, Gulshan-2, Dhaka 1212, Bangladesh',
  '211',
  'http://dbpe